<a href="https://colab.research.google.com/github/dfavenfre/Transfer-Learning-CNN-Tensorflow/blob/main/src/notebook/transfer_learning_cnn_model_fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# requirements.txt

In [ ]:
%pip install wandb -qU --quiet
%pip install tensorflow_hub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


## packages

In [ ]:
# Tensorflow Packages
import tensorflow as tf
import tensorflow_hub as hub
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPool2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout
    )

# WanDb Packages
from wandb.keras import (
    WandbMetricsLogger,
    WandbModelCheckpoint,
    WandbCallback
    )
import wandb

# Fundamental Packages
from typing import Callable, Optional, Dict, Tuple, Any, List
from datetime import datetime
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd
import logging
import numpy as np
import datetime
import random
import zipfile
import pprint
import pathlib
import json
import os
import math

# backend

## helper_functions.py

In [ ]:
def view_random_images(
    target_dir,
    target_class
    ):
  target_folder = os.path.join(target_dir, target_class)
  random_image = random.sample(os.listdir(target_folder), 1)
  img = mpimg.imread(target_folder + "/" + random_image[0])

  plt.imshow(img)
  # plt.title(target_class + " with a shape of: " + str(img.shape))
  plt.axis("off")

  return img

def resize_images(
    file_path: str,
    img_shape: int = 224
    ):
  image = tf.io.read_file(file_path)
  image = tf.image.decode_image(image)
  image = tf.image.resize(image,
                          size=[img_shape, img_shape]
                          )
  image = tf.expand_dims(image / 255., axis=0)

  return image


def unzip_data(file_path: str):
  zip_ref = zipfile.ZipFile(file_path)
  zip_ref.extractall()
  zip_ref.close()


def plot_predictions(
    target_dir: str,
    model
    ):
  class_labels = os.listdir("10_food_classes_all_data/train/")
  target_class = random.choice(class_labels)
  target_folder = os.path.join(target_dir, target_class)

  random_image = random.choice(os.listdir(target_folder))
  image_path = os.path.join(target_folder,
                            random_image)
  resized_image = resize_images(image_path)

  prediction = model.predict(resized_image)
  prediction_object = class_labels[int(tf.round(prediction))]

  image = mpimg.imread(image_path)
  plt.imshow(image)
  plt.title(f"Predicted Object: {prediction_object}")
  plt.axis(False)
  plt.show()


class ExpLRScheduler(Callback):
  def __init__(self, k: Optional[float] = 0.1):
    super(ExpLRScheduler, self).__init__()
    self.k = k

  def schedule_lr(self, epoch, lr):
    # Learning rate warm-up
    if epoch <= 8:
        return lr * math.exp((self.k * 0.125) * epoch)
    # LR exponential decay over k
    else:
        return lr * math.exp(-self.k (epoch / 512))

  def on_epoch_end(self, epoch, logs=None):
    updated_lr = self.schedule_lr(epoch, self.model.optimizer.lr.numpy())
    self.model.optimizer.lr.assign(updated_lr)
    print(f"*** Updated Learning Rate: {updated_lr} for epoch: {epoch + 1}")

class TrainingCheckPoint(Callback):
  def __init__(self, threshold: Optional[int] = 1, patience: Optional[int] = 5):
    super(TrainingCheckPoint, self).__init__()
    self.threshold = threshold
    self.patience = patience

  def on_epoch_end(self, epoch, logs):
    overfit_patience = 0
    overfit_ratio = logs["val_loss"] / logs["loss"]

    if self.threshold >= overfit_ratio:
      # self.model.save(f"model_{epoch}_{logs['val_accuracy']}.h5", overwrite=False)
      print(f"\ncurrent loss: {logs['loss']}\ncurrent validation loss: {logs['val_loss']}\n Epoch {epoch} was saved with {logs['val_accuracy']} accuracy")
    else:
      overfit_patience += 1
      print(f"Current overfitting epoch count {overfit_patience}")
      if overfit_patience >= self.patience:
        self.model.stop_training = True

def train_model_0() -> tf.keras.Model:

    EfficientNetV2B0_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
        include_top=False
        )
    EfficientNetV2B0_model.trainable=False
    input_layer = tf.keras.layers.Input(
    shape=(224,224,3),
    name="Input_Layer"
    )
    x = EfficientNetV2B0_model(input_layer)
    x = tf.keras.layers.GlobalAveragePooling2D(
        name="GAP2D_1"
    )(x)

    output_layer = tf.keras.layers.Dense(
        units=_OUTPUT_SIZE,
        activation="softmax",
        name="Ouput_Layer",
        kernel_regularizer=tf.keras.regularizers.L1()
    )(x)

    model_0 = tf.keras.Model(
        input_layer,
        output_layer,

        )

    return model_0


def optimize_network():

  global train_data
  global test_data
  global sweep_config

  tf.keras.backend.clear_session()
  with wandb.init(config=sweep_config):

    config = wandb.config

    if config.optimizers=="adam":
      optimizer = tf.keras.optimizers.Adam(
          learning_rate = config.lr
          )
    elif config.optimizers=="sgd":
      optimizer = tf.keras.optimizers.SGD(
          learning_rate=config.lr
      )
    else:
      raise NotImplementedError(
          f"Provided Optimizer :{config.optimizers} Not Implement"
          )
    model = train_model_0()
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(
            name="Categorical_Loss"
            ),
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    training_history = model.fit(
        train_data,
        validation_data=(test_data),
        epochs=config.epochs,
        batch_size=config.batch_size,
        verbose=1,
        callbacks=[
            WandbMetricsLogger(),
            ExpLRScheduler(k=config.k_value),
            TrainingCheckPoint(
                patience=config.early_stopping_patience)
        ]
    )


  return training_history


## preprocessing.py

In [ ]:
# Get data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
# Unzip it
unzip_data("/content/10_food_classes_10_percent.zip")

--2024-02-26 10:39:23--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.174.207, 74.125.23.207, 74.125.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.174.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  30.8MB/s    in 6.1s    

2024-02-26 10:39:29 (26.4 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
# CONSTANT VARIABLES
_IMG_SIZE = (224, 224)
_BATCH_SIZE = 32
_TRAINING_SET = "/content/10_food_classes_10_percent/train/"
_VALIDATION_SET = "/content/10_food_classes_10_percent/test/"
_OUTPUT_SIZE = 10

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=_TRAINING_SET,
    image_size=_IMG_SIZE,
    label_mode="categorical",
    labels="inferred",
    batch_size=_BATCH_SIZE,

)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=_VALIDATION_SET,
    image_size=_IMG_SIZE,
    label_mode="categorical",
    labels="inferred",
    batch_size=_BATCH_SIZE,

)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


## modelling.py

In [ ]:
sweep_config = {
    "method": "random",
    "name": "fv101 | EfficientNetV2B0_model_0 sweep",
    "metric": {"goal": "minimize", "name": "val_acc"},
    "parameters": {
        "optimizers": {"values": ["adam", "sgd"]},
        "batch_size": {"values": [8, 16, 32, 64]},
        "epochs": {"values": [300]},
        "lr": {"values": [1e-3, 1e-5, 1e-7]},
        "early_stopping_patience" : {"values": [10, 15]},
        "k_value" : {"values": [0.1, 0.09]}
    },
}

wandb.login()
sweep_id = wandb.sweep(sweep_config, project="FV 101 | Fine-Tuning ~ EfficientNetV2B0_model_0")

wandb: Currently logged in as: dfavenfre (alone-wolf). Use `wandb login --relogin` to force relogin


Create sweep with ID: nelfyd8x
Sweep URL: https://wandb.ai/alone-wolf/FV%20101%20%7C%20Fine-Tuning%20~%20EfficientNetV2B0_model_0/sweeps/nelfyd8x


In [ ]:
wandb.agent(sweep_id, function=optimize_network, count=20)

wandb: Agent Starting Run: 6wi51iy3 with config:
wandb: 	batch_size: 32
wandb: 	early_stopping_patience: 10
wandb: 	epochs: 300
wandb: 	k_value: 0.09
wandb: 	lr: 0.001
wandb: 	optimizers: sgd


24274472/24274472 [==============================] - 2s 0us/step
Epoch 1/300
24/24 [==============================] - ETA: 0s - loss: 6.7881 - accuracy: 0.0693*** Updated Learning Rate: 0.0010000000474974513 for epoch: 1

current loss: 6.788102626800537
current validation loss: 6.734729766845703
 Epoch 0 was saved with 0.07280000299215317 accuracy
24/24 [==============================] - 15s 222ms/step - loss: 6.7881 - accuracy: 0.0693 - val_loss: 6.7347 - val_accuracy: 0.0728
Epoch 2/300
23/24 [===========================>..] - ETA: 0s - loss: 6.7040 - accuracy: 0.1019*** Updated Learning Rate: 0.001011313567258426 for epoch: 2

current loss: 6.706332206726074
current validation loss: 6.656291484832764
 Epoch 1 was saved with 0.09200000017881393 accuracy
24/24 [==============================] - 3s 110ms/step - loss: 6.7063 - accuracy: 0.1013 - val_loss: 6.6563 - val_accuracy: 0.0920
Epoch 3/300
22/24 [==========================>...] - ETA: 0s - loss: 6.6346 - accuracy: 0.0966*** Updat

Traceback (most recent call last):
  File "<ipython-input-3-a30ba43fc8e9>", line 158, in optimize_network
    training_history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/execute.py", line 53, in quick_execute
    tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
Exception


epoch/accuracy,▁▂▄▆▆▇▇▇▇▇██████████████████████████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▆▆████▇▇▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▂▅▆▇▇▇█████████████████████████████████
epoch/val_loss,█▇▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.82133
epoch/epoch,218
epoch/learning_rate,2e-05
epoch/loss,2.54175
epoch/val_accuracy,0.8412


wandb: Agent Starting Run: 73gy1uwq with config:
wandb: 	batch_size: 32
wandb: 	early_stopping_patience: 10
wandb: 	epochs: 300
wandb: 	k_value: 0.09
wandb: 	lr: 1e-05
wandb: 	optimizers: sgd


...

Epoch 1/300
22/24 [==========================>...] - ETA: 0s - loss: 5.6591 - accuracy: 0.3764*** Updated Learning Rate: 0.0010000000474974513 for epoch: 1

current loss: 5.596941947937012
current validation loss: 4.420710563659668
 Epoch 0 was saved with 0.7071999907493591 accuracy
24/24 [==============================] - 12s 173ms/step - loss: 5.5969 - accuracy: 0.3867 - val_loss: 4.4207 - val_accuracy: 0.7072
Epoch 2/300
22/24 [==========================>...] - ETA: 0s - loss: 3.8089 - accuracy: 0.7060*** Updated Learning Rate: 0.00101257849963553 for epoch: 2

current loss: 3.768378257751465
current validation loss: 3.018458127975464
 Epoch 1 was saved with 0.8032000064849854 accuracy
24/24 [==============================] - 3s 109ms/step - loss: 3.7684 - accuracy: 0.7120 - val_loss: 3.0185 - val_accuracy: 0.8032
Epoch 3/300
23/24 [===========================>..] - ETA: 0s - loss: 2.6726 - accuracy: 0.8057*** Updated Learning Rate: 0.0010382120367098172 for epoch: 3

current loss: 

epoch/accuracy,▁▁▆█▇▅▅▅█▇▃▅▅▃▅█▄▃▄▄▃▄▅▇▅▅▃▅▃▃▄▅▄▅▅▅▅▅▅▆
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▆▇███▇▇▆▆▅▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▆▇█▇▇▅▅▅▄▄▃▃▂▂▃▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.82667
epoch/epoch,299
epoch/learning_rate,0.0
epoch/loss,1.30198
epoch/val_accuracy,0.8212


wandb: Agent Starting Run: z9rqupns with config:
wandb: 	batch_size: 32
wandb: 	early_stopping_patience: 15
wandb: 	epochs: 300
wandb: 	k_value: 0.09
wandb: 	lr: 0.001
wandb: 	optimizers: sgd


Epoch 1/300


wandb: Ctrl + C detected. Stopping sweep.
